#### Note. All data is simulated and fictitous, and is not real customer data

In [21]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd
import xlrd

%matplotlib inline

# I. Data field description

In [29]:
df_descrip_train = pd.read_excel("data/Data_Dictionary.xlsx",sheet_name='train')
df_descrip_history = pd.read_excel("data/Data_Dictionary.xlsx",sheet_name='history')
df_descrip_new_merchant_period = pd.read_excel("data/Data_Dictionary.xlsx",sheet_name='new_merchant_period')
df_descrip_merchant = pd.read_excel("data/Data_Dictionary.xlsx",sheet_name='merchant')

In [27]:
df_descrip_train.shape

(8, 2)

In [28]:
df_descrip_train

,train.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,card_id,Unique card identifier
3,first_active_month,"'YYYY-MM', month of first purchase"
4,feature_1,Anonymized card categorical feature
5,feature_2,Anonymized card categorical feature
6,feature_3,Anonymized card categorical feature
7,target,Loyalty numerical score calculated 2 months af...


In [30]:
df_descrip_history

,historical_transactions.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,card_id,Card identifier
3,month_lag,month lag to reference date
4,purchase_date,Purchase date
5,authorized_flag,"Y' if approved, 'N' if denied"
6,category_3,anonymized category
7,installments,number of installments of purchase
8,category_1,anonymized category
9,merchant_category_id,Merchant category identifier (anonymized )


In [31]:
df_descrip_new_merchant_period

,new_merchant_period.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,card_id,Card identifier
3,month_lag,month lag to reference date
4,purchase_date,Purchase date
5,authorized_flag,"Y' if approved, 'N' if denied"
6,category_3,anonymized category
7,installments,number of installments of purchase
8,category_1,anonymized category
9,merchant_category_id,Merchant category identifier (anonymized )


In [32]:
df_descrip_merchant

,merchants.csv,Unnamed: 1
0,NaN,NaN
1,Columns,Description
2,merchant_id,Unique merchant identifier
3,merchant_group_id,Merchant group (anonymized )
4,merchant_category_id,Unique identifier for merchant category (anony...
5,subsector_id,Merchant category group (anonymized )
6,numerical_1,anonymized measure
7,numerical_2,anonymized measure
8,category_1,anonymized category
9,most_recent_sales_range,Range of revenue (monetary units) in last acti...


# II. train.scv and test.csv

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [4]:
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
df_test.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


## Missing data

In [12]:
for col in df_train.columns:
    missing = df_train[col].isnull().sum()/df_train[col].shape[0]*100
    print("{:20}'s missing data:\t {}%".format(col,missing))

first_active_month  's missing data:	 0.0%
card_id             's missing data:	 0.0%
feature_1           's missing data:	 0.0%
feature_2           's missing data:	 0.0%
feature_3           's missing data:	 0.0%
target              's missing data:	 0.0%


In [13]:
for col in df_test.columns:
    missing = df_test[col].isnull().sum()/df_test[col].shape[0]*100
    print("{:20}'s missing data:\t {}%".format(col,missing))

first_active_month  's missing data:	 0.0008089109631702838%
card_id             's missing data:	 0.0%
feature_1           's missing data:	 0.0%
feature_2           's missing data:	 0.0%
feature_3           's missing data:	 0.0%


In [15]:
df_test.loc[df_test['first_active_month'].isnull()]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaN,C_ID_c27b4f80f7,5,2,1


* one missing data

In [16]:
del df_train, df_test

# III. Merchants.csv

In [17]:
df_merchants = pd.read_csv("data/merchants.csv")

In [18]:
df_merchants.head(5)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [19]:
df_merchants.shape

(334696, 22)

## Missing data check

In [33]:
for col in df_merchants.columns:
    missing = df_merchants[col].isnull().sum()/df_merchants[col].shape[0]*100
    print("{:20}'s missing data:\t {}%".format(col,missing))

merchant_id         's missing data:	 0.0%
merchant_group_id   's missing data:	 0.0%
merchant_category_id's missing data:	 0.0%
subsector_id        's missing data:	 0.0%
numerical_1         's missing data:	 0.0%
numerical_2         's missing data:	 0.0%
category_1          's missing data:	 0.0%
most_recent_sales_range's missing data:	 0.0%
most_recent_purchases_range's missing data:	 0.0%
avg_sales_lag3      's missing data:	 0.0038841217104476898%
avg_purchases_lag3  's missing data:	 0.0%
active_months_lag3  's missing data:	 0.0%
avg_sales_lag6      's missing data:	 0.0038841217104476898%
avg_purchases_lag6  's missing data:	 0.0%
active_months_lag6  's missing data:	 0.0%
avg_sales_lag12     's missing data:	 0.0038841217104476898%
avg_purchases_lag12 's missing data:	 0.0%
active_months_lag12 's missing data:	 0.0%
category_4          's missing data:	 0.0%
city_id             's missing data:	 0.0%
state_id            's missing data:	 0.0%
category_2          's missing da

In [36]:
print(df_merchants.shape[0]*0.000038841217104476898)
print(df_merchants.shape[0]*0.03551581136314745)

13.0
11887.0


In [38]:
df_merchants[['avg_sales_lag3','avg_sales_lag6','avg_sales_lag12']].loc[df_merchants['avg_sales_lag3'].isnull()]

,avg_sales_lag3,avg_sales_lag6,avg_sales_lag12
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN
11,NaN,NaN,NaN
12,NaN,NaN,NaN


* avg sales lag3,6,12 are missing in same people

# IV. new_merchant_transaction.csv

In [39]:
del df_merchants

In [40]:
df_new_merchants = pd.read_csv('data/new_merchant_transactions.csv')

In [41]:
df_new_merchants.shape

(1963031, 14)

In [42]:
df_new_merchants.head(5)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [43]:
for col in df_new_merchants.columns:
    missing = df_new_merchants[col].isnull().sum()/df_new_merchants[col].shape[0]*100
    print("{:20}'s missing data:\t {}%".format(col,missing))

authorized_flag     's missing data:	 0.0%
card_id             's missing data:	 0.0%
city_id             's missing data:	 0.0%
category_1          's missing data:	 0.0%
installments        's missing data:	 0.0%
category_3          's missing data:	 2.8487578647509895%
merchant_category_id's missing data:	 0.0%
merchant_id         's missing data:	 1.3354857870303627%
month_lag           's missing data:	 0.0%
purchase_amount     's missing data:	 0.0%
purchase_date       's missing data:	 0.0%
category_2          's missing data:	 5.692472508075522%
state_id            's missing data:	 0.0%
subsector_id        's missing data:	 0.0%


* at pd.read_csv
 1. nrows
 2. skiprows for kind of pagination
 3. iterator & chunksize

In [44]:
del df_new_merchants